In [ ]:
%config IPCompleter.greedy=True
import os
import copy
import subprocess
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import model_from_json
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

# utils.py imports
from utils import get_train_test_split, reshape_data, get_sample_weights
from utils import get_compiled_model #, eval_tr_model, eval_GS_v2, eval_te_data
from utils import eval_fo_te_data

In [ ]:
# Function set_seed:
    #Description:
        #The set_seed function sets the seed for random number generators in the numpy and tensorflow libraries. 
        #It takes an integer value as an argument that is used as the seed. The function sets the seed for the following 
        #random number generators:

        #    numpy.random.seed(): sets the seed for numpy random number generator.
        #    tf.random.set_seed(): sets the seed for tensorflow random number generator.
        #    tf.keras.seed: sets the seed for the keras module in tensorflow.
        #    os.environ['PYTHONHASHSEED']: sets the seed for python's hash function.
        #    os.environ['TF_DETERMINISTIC_OPS']: sets the flag to enforce deterministic behavior in tensorflow operations.

    # This function is useful for ensuring reproducibility of the results generated by machine learning models 
    # that use random number generators. By setting the seed, the same set of random numbers will be generated 
    # every time the model is run, ensuring that the results are consistent.

def set_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)  # Python general
    np.random.seed(seed)
    tf.random.set_seed(seed)
    tf.keras.seed = seed
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
# LSTM - Parameters
depths = [100, 200, 300]
layers = [2, 3]
drop_out_vals = [0.2, 0.4, 0.6]

# Data Import for training and testing
Note: the following description is how we implemented the code and data structure. Changes in this structure need to be applied in the code as well.

Data is imported using a pandas Dataframe which contains trajectory information and events extracted from ground reaction force (GRF) plates. The table is organized as follows (Note: column names are also used in the utils.py file - so when changing column names, also check the utils.py file!):

### 'Trial': 
    The name of each trial, represented as a string/char (e.g. 'DynamicA031.c3d').

### 'label':
    The label as an identifier for different pathologies, represented as an integer (e.g. 1,2,3, ... healthy, ICP, ...).
    
### 'DBid':	
    The unique identifier for each patient, represented as an integer.

### 'UsDat':	
    The capture date of the trial, represented as a datetime object.
    Note: with 'Trial', 'DBid', and 'USDat' each patient can be uniquely identified, even when there are more sessions.
    
### 'TR_TRAJ':
    A 2-dimensional numpy array (features, num_frames) with 36 rows and a varying number of columns 
    (depending on the length of the trial). 
    The first 18 columns represent a trajectory in the three dimensions (X, Y, Z) of a certain marker (HEEL, TOE, ANKLE). 
    Column 1-6 are the X-trajectories of the left HEEL, TOE, and ANKLE marker and X-trajectories of the right HEEL, TOE, 
    and ANKLE marker. Columns 7-13 contain the Y-trajectories and the remaining columns contain the Z-trajectories 
    (in the same order as before). 'TR_TRAJ' is also the input for the training of the neural network.
    The last 18 columns are the first derivaties of the above mentioned trajectories which represents 
    the velocity for each marker and axis. This leads to 36 features overall.
    The length before the first and after the last event should be randomized for each trial, so that the neural network
    learns different starting and ending points. In our paper we decided to use random values between 25 and 125. 

### 'TR_GRF_EVENTS':
    A 1-dimensional numpy array (num_frames) representing the ground truth for training. Each element in the array 
    corresponds to a time step and contains one of the following values: 0 (no event), 1 (left foot off), or 
    2 (right foot off). This array only contains events captured by a GRF plates (ground truth data). 
    'TR_GRF_EVENTS' is also the input for the training of the neural network.
    
### 'TE_TRAJ':
    A 2-dimensional numpy array (features, num_frames) that contains the same trajectories as 'TR_TRAJ', 
    but contains trajectories from the whole gait trial and not only for the GRF plate hits. 'TE_TRAJ' is used for 
    evaluating the performance of the beforehand trained model.
    
### 'TE_GRF_EVENTS': 
    A 1-dimensional numpy array (num_frames) that is the same length as 'TE_TRAJ' and contains only events from GRF
    plate hits. 'TE_GRF_EVENTS' is used to identify the performance of the beforehand trained model on
    ground truth data (=GRF plate hits). Note: our dataset was create with 0 = no event, 1 = left IC, 2 = left FO,
    3 = right IC, and 4 = right FO! This is important, because based on these labels, the validation is programmed.  
    

### #ALL_EVENTS':
    A 1-dimensional numpy array (num_frames) that contains is the same length as 'TE_TRAJ', but includes all events for
    the whole gait trial, including events not only from GRF plates. 
    Note: our dataset was create with 0 = no event, 1 = left IC, 2 = left FO, 3 = right IC, and 4 = right FO!  

In [ ]:
# imiport your DataFrame
oss_data = # ... import

In [ ]:
oss_data = read_mat('/home/projectdrive/nextcloud/p_2019_o3dga/OSS_DATA_random_start_end_v3_5.mat')
oss_data = pd.DataFrame(oss_data['OSS_DATA'])

In [ ]:
# creates a copy of the original data without a link
oss = pd.DataFrame(columns = oss_data.columns, data = copy.deepcopy(oss_data.values))

In [ ]:
# Further function description can be found in 'utils.py': 
# This function is used to split a given dataset into train, test, and validation sets. 
# It splits the data based on the labels present in the dataset and assigns a certain proportion of data to each set.
# Stratifies patients based on Trial ID. All trials from each patient are either in the training, validation or test split.
    # tr_data: This DataFrame contains the training data.
    # te_data: This DataFrame contains the test data.
    # tr_val_data: This DataFrame contains the training data for the validation set.
    # te_val_data: This DataFrame contains the test data for the validation set.
set_seed(5489) # Reproducibility!
hold_out = 0.3  # test split size
validation = 0.1 # validation split size
tr_data, te_data, tr_val_data, te_val_data = get_train_test_split(oss, hold_out, validation)

In [ ]:
# Further function description in 'utils.py':
# This function reshapes and pads sequences of trajectory and GRF data to the longest input sequence,
# so that they can be used as input for the neural network. The input shape of tr_traj is (features, num_frames).
# The input shape of tr_grf is (num_frames).
# Specifically, it pads the sequences with zeros so that all sequences have the same length 
# (the length of the longest sequence), and it reshapes the data to have the shape 
# (num_samples, max_seq_length, num_features) for tr_traj and (num_samples, num_frames) for tr_grf, 
# as this is the input format for the neural network.  

tr_val_traj, tr_val_grf = reshape_data(tr_val_data['TR_TRAJ'], tr_val_data['TR_GRF_EVENTS'])
val_traj, val_grf = reshape_data( te_val_data['TR_TRAJ'], te_val_data['TR_GRF_EVENTS'])
te_val_traj, te_val_grf = reshape_data(te_val_data['TE_TRAJ'], te_val_data['TE_GRF_EVENTS'])
te_traj, te_grf = reshape_data(te_data['TE_TRAJ'], te_data['TE_GRF_EVENTS']) # or te_data['ALL_EVENTS']

In [ ]:
# Definition of hyperparameters and early stopping meachenisms.
# Note: these are parameters from our best approach. They might not work for your data.
# 'min delta' depends a bit on the 'sample_weights', which drastically changes the 'val_loss' 
es = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=30, mode='auto', 
                   restore_best_weights=True, verbose=1, baseline=None)

# num of input features
features = 36

# num of output nodes
dense_out = 3

# num of hidden units of the LSTM layer
depth = 200

# num of stacked LSTM layers
layer = 3

# Droput layer rate in a range [0-1]
d_o_v = 0.4

# samples are HIGHLY imbalanced between no-events and events, therefore we use sample weights to 
# give more weight to rarely-seen events. A ratio of 1:10 (0.1 to 1) was found to be the best ratio.
weights = [
            [0.1, 1, 1, 1, 1],
            [0.01, 1, 1, 1, 1],
            [0.001, 1, 1, 1, 1]
          ]
weight = weights[0]


In [ ]:
tf.keras.backend.clear_session()
set_seed(5489)  # REPRODUCIBILITY

# Further function description in 'utils.py'.
# each frame of tr_val_grf is assigned a sample weight based on the label (no-event: 0, initial contact: 1, ...)
sample_weights = get_sample_weights(tr_val_grf, weight)

# Further function description in 'utils.py'
# creates the neural network with the given inputs (features, hidden units, dropout, layers, output nodes) 
# and compiles the model for training.
model = get_compiled_model(features, depth, d_o_v, layer, dense_out)

# Fit the model with the tr_val_traj and tr_val_grf data and validate eacht batch on the
# val_traj and val_grf data for performance (underfitting and overfitting) reasons.
history = model.fit(
    tr_val_traj,
    tf.one_hot(tr_val_grf, depth=3),
    epochs=200, batch_size=64, verbose=1,
    validation_data=(val_traj,tf.one_hot(val_grf, depth=3)),
    callbacks=[es],
    sample_weight=sample_weights,
    shuffle=True
)

In [ ]:
# Plot the val_loss (underfitting/overfitting)
from matplotlib import pyplot as plt
plt.plot(history.history['val_loss'])
plt.show()

In [ ]:
# This saves the model to the workspace
model.save('FO_production_model.h5')

In [ ]:
# This function can import a certain model and check for example the summmary()
model = tf.keras.models.load_model('FO_production_model.h5')
model.summary()

In [ ]:
# Further function description in 'utils.py':
# The function evaluates the performance of the input model on the test dataset. 
# It calculates the mean absolute error for each event between prediction and GRF events 
# for each pathology separately (labels).
# Note: This should only be used to validate the FINAL model!
# this can be used for the validation set to find the best hyperparameters in a grid search or
# Bayesian Optimization using the KerasTuner: https://keras.io/keras_tuner/

fo_mae, fo_list_all, foclassification = eval_fo_te_data(model, te_traj, te_grf, te_data['label'])